# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [6]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = pd.read_csv('../../05_src/data/adult/adult.data', header=None, names=columns).assign(income=lambda x: (x.income.str.strip() == '>50K')*1)


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [7]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop('income', axis=1)  # All columns except 'income'
Y = adult_dt['income']               # Only the 'income' column

# Split the data into training and testing sets (70-30 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

Random state of the splitting function is the one that is used in train and test splits and cross functional validation in ML. It refers to a seed value for the random number generator. This seed ensures that the results are reproducible.It is useful in maintaining consistency and reliability in machine learning experiments.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [9]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline  # Importing Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
data = pd.DataFrame({
    'numerical_1': [1, 2, np.nan, 4, 5],
    'numerical_2': [np.nan, 2, 3, 4, 5],
    'categorical_1': ['A', 'B', 'A', np.nan, 'B'],
    'categorical_2': [np.nan, 'X', 'Y', 'X', 'Y']
})

# Preprocessing steps
numerical_features = ['numerical_1', 'numerical_2']
categorical_features = ['categorical_1', 'categorical_2']

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')), 
             ('scaler', RobustScaler())
         ]), 
         numerical_features),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')), 
             ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_features)
    ]
)

# Example of fitting the transformer on the data
preprocessed_data = preprocessor.fit_transform(data)

# Convert the result back to a DataFrame if needed
preprocessed_df = pd.DataFrame(preprocessed_data)
print(preprocessed_df)


     0         1    2    3
0 -1.2 -0.095238  0.0  0.0
1 -0.7 -0.904762  1.0  0.0
2  0.0  0.000000  0.0  1.0
3  0.3  0.904762  0.0  0.0
4  0.8  1.809524  1.0  1.0


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [10]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline  # Importing Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier

data = pd.DataFrame({
    'numerical_1': [1, 2, np.nan, 4, 5],
    'numerical_2': [np.nan, 2, 3, 4, 5],
    'categorical_1': ['A', 'B', 'A', np.nan, 'B'],
    'categorical_2': [np.nan, 'X', 'Y', 'X', 'Y'],
    'target': [0, 1, 0, 1, 0]  # Example target variable for classification
})

# Define features and target
X = data.drop('target', axis=1)  
y = data['target']                

# Define the preprocessing steps
numerical_features = ['numerical_1', 'numerical_2']
categorical_features = ['categorical_1', 'categorical_2']

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')), 
             ('scaler', RobustScaler())
         ]), 
         numerical_features),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')), 
             ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_features)
    ]
)
# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  
    ('classifier', RandomForestClassifier())  
model_pipeline.fit(X, y)
# model_pipeline for predictions or evaluation
predictions = model_pipeline.predict(X)
print(predictions)


[0 1 0 1 0]


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline

# Define numerical and categorical features
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
             ('scaler', RobustScaler())
         ]), 
         numerical_features),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),
             ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_features)
    ]
)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])


In [22]:
from sklearn.model_selection import StratifiedKFold, cross_validate

# Set up stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5)

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation
cv_results = cross_validate(model_pipeline, X_train, Y_train, cv=skf, scoring=scoring, return_train_score=True)

# Reporting the results
print("Training Results:")
print(f"Negative Log Loss: {np.mean(cv_results['train_neg_log_loss'])}")
print(f"ROC AUC: {np.mean(cv_results['train_roc_auc'])}")
print(f"Accuracy: {np.mean(cv_results['train_accuracy'])}")
print(f"Balanced Accuracy: {np.mean(cv_results['train_balanced_accuracy'])}")

print("\nValidation Results:")
print(f"Negative Log Loss: {np.mean(cv_results['test_neg_log_loss'])}")
print(f"ROC AUC: {np.mean(cv_results['test_roc_auc'])}")
print(f"Accuracy: {np.mean(cv_results['test_accuracy'])}")
print(f"Balanced Accuracy: {np.mean(cv_results['test_balanced_accuracy'])}")





/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Results:
Negative Log Loss: -0.08135273731172007
ROC AUC: 0.9999999967246861
Accuracy: 0.9999561247154632
Balanced Accuracy: 0.9999095329627693

Validation Results:
Negative Log Loss: -0.3561276508190201
ROC AUC: 0.9037682528650983
Accuracy: 0.8538088834019684
Balanced Accuracy: 0.7761287853181011


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [23]:
import pandas as pd
cv_results_df = pd.DataFrame(cv_results)

# Sort by negative log loss in descending order
cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)

# Display the sorted results
cv_results_df_sorted


,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
3,5.786308,0.200097,-0.336616,-0.081675,0.906551,1.0,0.860026,0.999945,0.782388,0.999887
1,5.787002,0.201691,-0.353602,-0.080846,0.903076,1.0,0.845361,0.999945,0.765478,0.999887
2,5.745641,0.199652,-0.360349,-0.081139,0.902561,1.0,0.855638,1.000000,0.779184,1.000000
0,6.019570,0.216389,-0.365017,-0.081921,0.903450,1.0,0.850186,0.999945,0.776038,0.999887
4,6.029034,0.215567,-0.365055,-0.081182,0.903204,1.0,0.857832,0.999945,0.777555,0.999887


Calculate the mean of each metric. 

In [25]:
# Calculate the mean of each metric
metrics_mean = {
    'train_neg_log_loss': cv_results['train_neg_log_loss'].mean(),
    'test_neg_log_loss': cv_results['test_neg_log_loss'].mean(),
    'train_roc_auc': cv_results['train_roc_auc'].mean(),
    'test_roc_auc': cv_results['test_roc_auc'].mean(),
    'train_accuracy': cv_results['train_accuracy'].mean(),
    'test_accuracy': cv_results['test_accuracy'].mean(),
    'train_balanced_accuracy': cv_results['train_balanced_accuracy'].mean(),
    'test_balanced_accuracy': cv_results['test_balanced_accuracy'].mean()
}

# Display the mean metrics
metrics_mean_df = pd.DataFrame([metrics_mean])
metrics_mean_df


,train_neg_log_loss,test_neg_log_loss,train_roc_auc,test_roc_auc,train_accuracy,test_accuracy,train_balanced_accuracy,test_balanced_accuracy
0,-0.081353,-0.356128,1.0,0.903768,0.999956,0.853809,0.99991,0.776129


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [27]:
# Refit the model pipeline on the entire training data
model_pipeline.fit(X_train, Y_train)

# Get prediction probabilities for X_test
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate the performance metrics using Y_pred_proba for log loss and roc_auc
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),  # Negative log loss
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),  # Only the positive class probabilities
    'accuracy': accuracy_score(Y_test, model_pipeline.predict(X_test)),
    'balanced_accuracy': balanced_accuracy_score(Y_test, model_pipeline.predict(X_test))
}
test_metrics


{'neg_log_loss': -0.38178311500088813,
 'roc_auc': 0.9010240353095236,
 'accuracy': 0.8549493295117208,
 'balanced_accuracy': 0.7754760484543678}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

It is convinient because it easier to work with for binary classification tasks. By transforming the target variable immediately, we simplify thr subsequent steps, avoiding the need to reformat it later and reducing the risk of errors.It simplifies the target into binary format.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.